In [2]:
from IPython.display import Image
import torch

- FSDP: Fully Sharded Data Parallel, by Facebook;
    - fsdp unit & sharding
        - fsdp unit：model parallel
        - sharding：os + g + p
    - https://docs.google.com/presentation/d/1ntPSYg-Wphl8sErwjUl0AztOY1i4SZmQuvmGhkeRElA/edit#slide=id.g2318fd43235_0_292
- 通过这次的 tutorial 再整体回顾下整个系列关于分布式的基本概念/术语，以及方法；

## GPU memory

- Shard parameters, gradients, and optimizer states across all data-parallel processes
    - GPU memory:
        - P: Parameters
        - G: Gradients
        - OS: Optimizer states
    - 暂不考虑 features/activations/embeddings
    - 都跟 optimizer 有关
        - 优化器的构造会封装 parameters：`optimizer = optim.Adam(model.parameters(), lr=0.001)`
        - loss.backward() => parameters.grad
        - optimizer.step() => optimizer states
            - momentum：gradient 的指数平均
            - variance：gradient square 的指数平均


```
for group in optimizer.param_groups:
    for p in group['params']:
        state = optimizer.state[p]

        # Exponential moving average of gradient values
        m = state['exp_avg']  # 动量参数

        # Exponential moving average of squared gradient values
        v = state['exp_avg_sq']  # 方差参数
```

- 混合精度下的 GPU memory 占用，$x$ 个模型参数（fp16）
    - Parameters：$2x$
    - Gradients：$2x$
    - Optimizer states (Adam, all is fp32) : $12x = 4x + 4x + 4x$
        - Parameters copy：$4x$
        - Momentum：$4x$
        - Variance:：$4x$
    - 参考 https://arxiv.org/abs/1910.02054（ZeRO: Memory Optimizations Toward Training Trillion Parameter Models）
        - ZeRO：Zero Redundancy Optimizer (ZeRO)

In [7]:
# k=12
Image(url='https://www.microsoft.com/en-us/research/uploads/prod/2020/02/DeepSpeed-Image-1.png', width=600)

In [5]:
# https://pytorch.org/docs/stable/generated/torch.optim.Adam.html
Image(url='../imgs/adam.png', width=500)

## DDP => FSDP

The NVIDIA Collective Communication Library (NCCL) implements multi-GPU and multi-node **communication primitives** optimized for NVIDIA GPUs and Networking.
NCCL provides routines such as 

- all-gather,
- all-reduce,
- broadcast,
- reduce,
- reduce-scatter
- as well as point-to-point send and receive

that are optimized to achieve high bandwidth and low latency 

- over PCIe and NVLink high-speed interconnects within a node
- over NVIDIA Mellanox Network across nodes.
    - InfiniBand: IB，无限带宽，集群互联；（对应的是 ethernet 以太网）
    - Mellanox 主要是做 IB 的，2019年被 Nvidia 收购；加速计算与互联/存储的结合；

In [8]:
Image(url='../imgs/ddp/ddp_allreduce.png', width=500)

For every GPU (node)
- FeedForward **locally**
- Backward & compute gradient **locally**
- **AllReduce(gradient) – across nodes**
    - (DDP) training, each process/ worker owns a replica of the model and processes a batch of data, finally it uses all-reduce to sum up gradients over different workers. 
- Update optimizer states and weights **locally**

What is Not FSDP
- Model Parallelism：模型的不同的 layer 放在不同的 gpu 上；
- Tensor Parallelism：分块矩阵实现
    - split matrix multiplication by Column
- Pipeline Parallelism
    - Mix Data and Model Parallelism

### tensor parallelism

In [35]:
Image(url='../imgs/ddp/fsdp_column.png', width=500)

In [21]:
A = torch.arange(1, 7).reshape(2, 3).to(torch.float)
B = torch.arange(1, 7).reshape(3, 2).to(torch.float)
A, B, A@B

(tensor([[1., 2., 3.],
         [4., 5., 6.]]),
 tensor([[1., 2.],
         [3., 4.],
         [5., 6.]]),
 tensor([[22., 28.],
         [49., 64.]]))

In [32]:
B1 = B[:, 0].view(-1, 1)
B2 = B[:, 1].view(-1, 1)

In [33]:
A @ B1, A @ B2

(tensor([[22.],
         [49.]]),
 tensor([[28.],
         [64.]]))

In [34]:
torch.concat([A@B1, A@B2], dim=1)

tensor([[22., 28.],
        [49., 64.]])

### pipeline parallel

In [39]:
# Model Parallelism using multiple GPUs
Image(url='https://pytorch.org/docs/stable/_images/no_pipe.png', width=400)

- The figure represents a model with 4 layers placed on 4 different GPUs (vertical axis). 
- The horizontal axis represents training this model through time demonstrating that only 1 GPU is utilized at a time
- 任何时刻，只有一张卡在做计算；

In [40]:
# Pipelined Execution
Image(url='https://pytorch.org/docs/stable/_images/pipe.png', width=400)

- $F_{i,j}$
    - $i$ 表示 gpu card，model parts
    - $j$ 表示 data splits
- To alleviate this problem, pipeline parallelism splits the input minibatch into multiple microbatches and pipelines the execution of these microbatches across multiple GPUs.
- The figure represents a model with 4 layers placed on 4 different GPUs (vertical axis). The horizontal axis represents training this model through time demonstrating that the GPUs are utilized much more efficiently. However, there still exists a bubble (as demonstrated in the figure) where certain GPUs are not utilized. 

```
from torch.distributed.pipeline.sync import Pipe

# Need to initialize RPC framework first.
os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '29500'
torch.distributed.rpc.init_rpc('worker', rank=0, world_size=1)

# Build pipe.
fc1 = nn.Linear(16, 8).cuda(0)
fc2 = nn.Linear(8, 4).cuda(1)
model = nn.Sequential(fc1, fc2)
# chunks: number of micro-batches (default: 1)
model = Pipe(model, chunks=8)

input = torch.rand(16, 16).cuda(0)
output_rref = model(input)
```

### fsdp

- PyTorch FSDP: Experiences on Scaling Fully Sharded Data Parallel
    - https://arxiv.org/pdf/2304.11277
- 流程
    - FSDP Unit [Vertically “Splitting”]
        - layer/module/stage
    - Sharding [Horizontally “Splitting”]
        - os + g + p
    - All-Gather
    - Reduce-Scatter
- split our FSDP-Unit parameters across GPUs
- all-gather per FSDP-unit => Forward/Backward

In [5]:
Image(url='../imgs/ddp/fsdp_overall.png', width=500)

- construct units
    - unit 0: [layer 0, layer 3]
        - modules that share parameters must be wrapped as part of the same FSDP unit. 
    - unit 1: [layer 1, layer 2]
    - unit 2: [layer 4, layer 5]
- sharding
    - store fsdp unit on `FlatParameter`
    - split `FlatParameter` on multiple nodes
    - `torch.distributed.fsdp.FullyShardedDataParallel` (https://pytorch.org/docs/stable/fsdp.html)
        - `sharding_strategy`:
            - FULL_SHARD: os + g + p;
            - SHARD_GRAD_OP: os + g;

In [6]:
Image(url='../imgs/ddp/unit_sharding.png', width=500)

- All-Gather
    - gather (concat) + broadcast
- Reduce-Scatter

In [3]:
# all gather
Image(url='https://docs.nvidia.com/deeplearning/nccl/user-guide/docs/_images/allgather.png', width=500)

In [8]:
Image(url='../imgs/ddp/fsdp_allgather.png', width=500)

In [10]:
# 0, 1, 2: 分别表示不同的 unit
Image(url='../imgs/ddp/overlap_comm_comp.png', width=500)

In [12]:
# reduce-scatter
Image(url='https://docs.nvidia.com/deeplearning/nccl/user-guide/docs/_images/reducescatter.png', width=500)

In [4]:
Image(url='https://pytorch.org/tutorials/_images/fsdp_sharding.png', width=500)

In [14]:
# difference minibatches => different gradients
Image(url='../imgs/ddp/fsdp_red_scatter.png', width=500)

In [16]:
# Train Billion-size Models
# More communication between GPUs
# Trade memory for time
Image(url='../imgs/ddp/ddp_fsdp.png', width=500)

## torch api

```
import torch
from torch.distributed._fsdp import FullyShardedDataParallel as FSDP

torch.cuda.set_device(device_id)

sharded_module = FSDP(my_module)
optim = torch.optim.Adam(sharded_module.parameters(), lr=0.0001)
sharded_module(input).sum().backward()
optim.step()
```